<a href="https://colab.research.google.com/github/ckqqqq/Kaggle_Covid_19_NER/blob/main/Kaggle_Covid_19_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip3 install torch==1.8.2+cu102 torchvision==0.9.2+cu102 torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/Kaggle

/content/drive/MyDrive/Colab Notebooks/Kaggle


In [ ]:
#!pip install pycuda

In [ ]:
# Auchor Kaggle 
# Keqi Chen 
# tianqingfang https://www.kaggle.com/tianqingfang/playground Standard inputoutput
# Pytorch Document :BILSTM
import torch
import torch.nn as nn
import torch.optim as optim
import pickle as pkl
from itertools import chain
import pandas as pd
#import pycuda#
import keras
import numpy as np
import os
#core
torch.manual_seed(1)
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 32
HIDDEN_DIM = 32
EPOCH = 1
gpu_use=torch.cuda.is_available()
def argmax(vec):
    # return the argmax 
    _, idx = torch.max(vec, 1)
    return idx.item()

 # Transferm seq_lest to Seq_tesor
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long) 


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))


# main model
class BiLSTM_CRF(nn.Module):
    # vocabulary size| mag_(label-number)| embedding dimension| hidden dimension
    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.gpu_use=torch.cuda.is_available()
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)
        

        # "//"
        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # transition parameters  Entry i,j is the score of
        # transitioning *to* i *from* j.
        # initialization
        self.transitions = nn.Parameter(torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        # initial the Start and Tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()
        if self.gpu_use:#accerlerate
          #print("GPU")
          self.lstm.cuda()
          self.hidden2tag.cuda()
          self.transitions.cuda()
        

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the *forward algorithm* to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        if self.gpu_use:#accerlerate
          #print("GPU)
          init_alphas.cuda()
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
            if self.gpu_use:#accerlerate
              #print("GPU)
              forward_var.cuda()
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                    self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score
    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        #
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats
    #dijkstrbi
    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  
            # holds the backpointers for this step
            viterbivars_t = []  
            # holds the viterbi variables for this step
            # https://www.zhihu.com/question/20136144
            # 233333 Like dijkstra algorithm

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        # predict
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check ??
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  
        # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features. 
        score, tag_seq = self._viterbi_decode(lstm_feats)
        # Explanation https://space.bilibili.com/88461692/channel/detail?cid=26587
        return score, tag_seq

In [ ]:
#
path =r"/content/drive/MyDrive/Colab Notebooks/Kaggle/"
train_dict = pkl.load(open(path+r"/Kaggle_input/train.pkl", "rb"))
val_dict = pkl.load(open(path+r"/Kaggle_input/val.pkl", "rb"))
test_dict = pkl.load(open(path+r"/Kaggle_input/test.pkl", "rb"))
print("keys in train_dict:", train_dict.keys())
print("keys in val_dict:", val_dict.keys())
print("keys in test_dict:", test_dict.keys())
print("count of the NER tags:", len(set(chain(*train_dict["tag_seq"]))))
# print("all the NER tags:", set(chain(*train_dict["tag_seq"])))
# prepare word vocab and tag vocab

vocab_dict = {'_unk_': 0, '_w_pad_': 1}
# vocab
for doc in train_dict['word_seq']:
    for word in doc:
        if (word not in vocab_dict):
            vocab_dict[word] = len(vocab_dict)
# attention [0]
tag_dict = {'_t_pad_': 0}  
# add a padding token
# tags
for tag_seq in train_dict['tag_seq']:
    for tag in tag_seq:
        if (tag not in tag_dict):
            tag_dict[tag] = len(tag_dict)
word2idx = vocab_dict
idx2word = {v: k \
             for k, v in word2idx.items()}# reverse！！！review Note
# print(idx2word)# too many
tag2idx = tag_dict
idx2tag = {v: k \
            for k, v in tag2idx.items()} #reverse！！！1111111
tag2idx[START_TAG] = len(tag2idx)
tag2idx[STOP_TAG] = len(tag2idx)
print(tag2idx)

print("size of word vocab:", len(vocab_dict), "size of tag_dict:", len(tag_dict))
print("Model initialized")
# model = BiLSTM_CRF(len(word2idx), tag2idx, EMBEDDING_DIM, HIDDEN_DIM)

# The maximum length of a sentence is set to 128
max_sent_length = 128

train_tokens = np.array([[word2idx[w] \
                          for w in doc] \
                         for doc in train_dict['word_seq']])
# for test tp prevent overfit
val_tokens = np.array([[word2idx.get(w, 0)  \
                        for w in doc] \
                       for doc in val_dict['word_seq']])
# Useless in this question



train_tags = [[tag2idx[t] \
               for t in t_seq] \
              for t_seq in train_dict['tag_seq']]
#transfrom to binary file
train_tags = np.array([to_categorical(t_seq, num_classes=len(tag_dict)) \
                       for t_seq in train_tags])

val_tags = [[tag2idx[t] \
             for t in t_seq] \
            for t_seq in val_dict['tag_seq']]

# val_tags = np.array([to_categorical(t_seq, num_classes=len(tag_dict)) \
#                      for t_seq in val_tags])

# we don't have test tags
print(val_tags[0])
print(train_tags[0])



keys in train_dict: dict_keys(['id', 'word_seq', 'tag_seq'])
keys in val_dict: dict_keys(['id', 'word_seq', 'tag_seq'])
keys in test_dict: dict_keys(['id', 'word_seq'])
count of the NER tags: 65
{'_t_pad_': 0, 'O': 1, 'LIVESTOCK': 2, 'DISEASE_OR_SYNDROME': 3, 'GENE_OR_GENOME': 4, 'CARDINAL': 5, 'CHEMICAL': 6, 'PRODUCT': 7, 'QUANTITY': 8, 'NORP': 9, 'THERAPEUTIC_OR_PREVENTIVE_PROCEDURE': 10, 'CELL': 11, 'ORGANISM': 12, 'GROUP': 13, 'ORDINAL': 14, 'GPE': 15, 'ORG': 16, 'LABORATORY_PROCEDURE': 17, 'DATE': 18, 'CORONAVIRUS': 19, 'EUKARYOTE': 20, 'SIGN_OR_SYMPTOM': 21, 'VIRUS': 22, 'CELL_COMPONENT': 23, 'MOLECULAR_FUNCTION': 24, 'CELL_OR_MOLECULAR_DYSFUNCTION': 25, 'VIRAL_PROTEIN': 26, 'HUMAN-CAUSED_PHENOMENON_OR_PROCESS': 27, 'BODY_PART_ORGAN_OR_ORGAN_COMPONENT': 28, 'PERSON': 29, 'TISSUE': 30, 'RESEARCH_ACTIVITY': 31, 'EVENT': 32, 'IMMUNE_RESPONSE': 33, 'ORGAN_OR_TISSUE_FUNCTION': 34, 'MATERIAL': 35, 'EVOLUTION': 36, 'LABORATORY_OR_TEST_RESULT': 37, 'BACTERIUM': 38, 'MONEY': 39, 'FAC': 40

In [ ]:
test_tokens = np.array([[word2idx.get(w, 0) \
                         for w in doc] \
                        for doc in test_dict['word_seq']])

In [ ]:


#accerlerate
if gpu_use :
  # pycuda.cuda.init()
  
  # torch.load('modelparameters.pth', map_location=lambda storage, loc: storage.cuda(0))

  print("Cuda:Version",torch.version.cuda)
  print(torch.cuda.current_device())
  # num = cuda.Device.count()
  # print("%d device(s) found:"%num)
  # for i in range(num):
  #   print(cuda.Device(i).name(), "(Id: %d)"%i)
else:
  print("Warnning No Cuda ！setting_ GPU")

Warnning No Cuda ！setting_ GPU


In [ ]:
print("model initailization:\n")
model = BiLSTM_CRF(len(word2idx), tag2idx, EMBEDDING_DIM, HIDDEN_DIM)
if gpu_use :#accerlerate 
  model.cuda()

model initailization:



In [ ]:


#https://blog.csdn.net/xiaosongshine/article/details/89401522
#https://blog.csdn.net/w113691/article/details/106571007/
   

# optimize
optimizer = optim.SGD(model.parameters(), lr=0.02, weight_decay=1e-4)
# We are short on GPU

#Check predictions before training
# with torch.no_grad():
#     precheck_sent = prepare_sequence(val_tokens, word2idx)
#     precheck_tags = torch.tensor([tag2idx[t] for t in val_tags], dtype=torch.long)
#     print(model(precheck_sent))
print("Training beginning\n")
# Make sure prepare_sequence from earlier in the LSTM section is loaded
EPOCH=5
#28000
#
for ep in range(EPOCH):  # 
    Counter =0
    for sentence_sqe, tags_sqe in zip(val_tokens,val_tags):
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()


        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        # sentence_in = prepare_sequence(sentence, word2idx)

        sentence_tor = torch.tensor(sentence_sqe, dtype=torch.long)
        tag_tor = torch.tensor(tags_sqe, dtype=torch.long)
        if gpu_use:#accerlerate
          #print("GPU")
          sentence_tor.cuda()
          tag_tor.cuda()

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_tor, tag_tor)
      
        
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()
        Counter += 1
        if Counter%10==0:
          print(Counter,"Loss: ",loss)
        if Counter==400:
          break

        #128*128
        #128*128 2800 val 28000 train *100000
        
print("Training finished!!!!!:")


# We got it!
#Check predictions after training
predict=[]
Counter=0
with torch.no_grad():
  print(model(torch.tensor(val_tokens[1],dtype=torch.long)))
with torch.no_grad():
  print(torch.tensor(val_tokens,dtype=torch.long))
  for val_token in val_tokens:
    predict.append(model(torch.tensor(val_token,dtype=torch.long))[1])  
    

print("Predic type:", type(predict))

val_preds = np.array([[idx2tag[p] for p in pred_tor] for pred_tor in predict])
print(val_preds[2])  

Training beginning

10 Loss:  tensor([195.9117], grad_fn=<SubBackward0>)
20 Loss:  tensor([450.0818], grad_fn=<SubBackward0>)
30 Loss:  tensor([218.0327], grad_fn=<SubBackward0>)
40 Loss:  tensor([166.6262], grad_fn=<SubBackward0>)
50 Loss:  tensor([150.3998], grad_fn=<SubBackward0>)
60 Loss:  tensor([162.1233], grad_fn=<SubBackward0>)
70 Loss:  tensor([93.0287], grad_fn=<SubBackward0>)
80 Loss:  tensor([185.0294], grad_fn=<SubBackward0>)
90 Loss:  tensor([171.3558], grad_fn=<SubBackward0>)
100 Loss:  tensor([139.9319], grad_fn=<SubBackward0>)
110 Loss:  tensor([110.7762], grad_fn=<SubBackward0>)
120 Loss:  tensor([147.6701], grad_fn=<SubBackward0>)
130 Loss:  tensor([136.1213], grad_fn=<SubBackward0>)
140 Loss:  tensor([50.1854], grad_fn=<SubBackward0>)
150 Loss:  tensor([146.6354], grad_fn=<SubBackward0>)
160 Loss:  tensor([138.0318], grad_fn=<SubBackward0>)
170 Loss:  tensor([36.8644], grad_fn=<SubBackward0>)
180 Loss:  tensor([168.7064], grad_fn=<SubBackward0>)
190 Loss:  tensor([2

In [ ]:
with torch.no_grad():
  print(model(torch.tensor(val_tokens[5],dtype=torch.long))[1])                 
  print(model)
print(val_preds[2]) 

print(len(val_preds))
print(val_preds.size)

print(len(val_dict["id"])) 




[1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 1, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 10, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 1, 3, 1, 1, 1, 6, 1, 1, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 3, 3, 1, 3, 3, 13, 1, 1, 22, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1]
BiLSTM_CRF(
  (word_embeds): Embedding(82275, 32)
  (lstm): LSTM(32, 16, bidirectional=True)
  (hidden2tag): Linear(in_features=32, out_features=67, bias=True)
)
['GENE_OR_GENOME' 'GENE_OR_GENOME' 'GENE_OR_GENOME' 'O'
 'DISEASE_OR_SYNDROME' 'DISEASE_OR_SYNDROME' 'DISEASE_OR_SYNDROME'
 'DISEASE_OR_SYNDROME' 'O' 'GENE_OR_GENOME' 'GENE_OR_GENOME'
 'GENE_OR_GENOME' 'O' 'O' 'O' 'GENE_OR_GENOME' 'GENE_OR_GENOME'
 'GENE_OR_GENOME' 'O' 'DISEASE_OR_SYNDROME' 'O' 'O' 'O' 'O' 'O' 'O' 'O'
 'O' 'O' 'O' 'O' 'DISEASE_OR_SYNDROME' 'GROUP' 'O' 'O' 'O' 'O' 'O' 'O' 'O'
 'O' 'O' 'O' 'GENE_OR_GENOME' 'GENE_OR_GENOME' 'O' 'O' 

In [ ]:

#f_dict
def get_pred_csv(pred, f_dict, padding_id="_w_pad_"):
    ids = [str(i)+"_"+str(w_id) for i, _ in enumerate(f_dict["id"]) for w_id, word in enumerate(f_dict["word_seq"][i]) if word != padding_id]
    tags = [ps[w_id] for i, ps in enumerate(pred) for w_id, word in enumerate(f_dict["word_seq"][i]) if word != padding_id]
    return {"id":ids, "tag":tags}
#accuracy(D1,D2)
def get_csv_accuracy(d1, d2):
    assert len(d1["id"]) == len(d2["id"])
    return sum(np.array(d1["tag"]) == np.array(d2["tag"])) / len(d1["tag"])
    
val_pred_dict = get_pred_csv(val_preds, val_dict)
# Calculate acuracy rate
print(len(val_pred_dict["id"])) 
print(len(pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Kaggle/Kaggle_input/val_labels.csv")))
print("validation acc", get_csv_accuracy(val_pred_dict, pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Kaggle/Kaggle_input/val_labels.csv")))
pd.DataFrame(val_pred_dict).to_csv("val_pred_final.csv", index=False)

345690
345690
validation acc 0.7696780352338801


In [ ]:
predict=[]
Counter=0
with torch.no_grad():
  print(model(torch.tensor(test_tokens[1],dtype=torch.long)))
with torch.no_grad():
  print(torch.tensor(test_tokens,dtype=torch.long))
  for test_token in test_tokens:
    predict.append(model(torch.tensor(test_token,dtype=torch.long))[1])  
    

print("Predic type:", type(predict))

test_preds = np.array([[idx2tag[p] for p in pred_tor] for pred_tor in predict])
print(test_preds[2]) 
test_preds_dict = get_pred_csv(test_preds, test_dict)
print(len(test_preds_dict["id"])) 
pd.DataFrame(test_preds_dict).to_csv("test_pred_final.csv", index=False)

(tensor(1487.7362), [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([[    0,  2372,  1144,  ...,   358,     3,     0],
        [   50,  3185,     3,  ...,     1,     1,     1],
        [17744,   561,   106,  ...,    55, 10061,   106],
        ...,
        [ 4639,     3, 21515,  ...,  6100,     3,  1218],
        [ 4580, 11637,     0,  ...,  1836,    96,  6890],
        [37233,   106,  6983,  ...,    51, 33627,    61]])
Predic type: <class 'list'>
['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'CHEMICAL' 'CHEMICAL' 'O' 'O' 'O' 'O'
 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O'
 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'CHEMICAL' 'CHE

In [ ]:
print(len(test_preds_dict["id"])) 
pd.DataFrame(test_preds_dict).to_csv("submission2.csv", index=False)

349105


In [ ]:
torch.save(model, '/content/drive/MyDrive/Colab Notebooks/Kaggle/model.pt')

https://www.cnblogs.com/xiximayou/p/12450761.html

In [ ]:
state = { 'model': model.state_dict(),'optimizer':optimizer.state_dict(),'epoch':EPOCH}   
torch.save(state,'/content/drive/MyDrive/Colab Notebooks/Kaggle/model2.pt')


In [ ]:

test_preds_dict = get_pred_csv(test_preds, test_dict)
print(len(test_preds_dict["id"])) 
pd.DataFrame(test_preds_dict).to_csv("test_predfinal.csv", index=False)

2791221


In [ ]:
# Please submit this file
pd.read_csv("train_predfinal.csv")

,id,tag
0,0_0,O
1,0_1,O
2,0_2,O
3,0_3,O
4,0_4,O
...,...,...
2791216,23599_123,O
2791217,23599_124,O
2791218,23599_125,O
2791219,23599_126,O


In [ ]:
# Please submit this file
pd.read_csv("val_pred.csv")

,id,tag
0,0_0,O
1,0_1,O
2,0_2,O
3,0_3,O
4,0_4,O
...,...,...
345685,2949_123,O
345686,2949_124,O
345687,2949_125,O
345688,2949_126,O
